# Log-Sum-Exp

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from scipy import stats

In [ ]:
# 混合ガウス分布のEMアルゴリズム

K = 2 # 混合ガウス分布の数（固定）

def scale(X):
    """
    データ行列X(サンプル数、特徴数)のフィールド(カラム列)ごとに標準化
    """
    
    col = X.shape[1]
    
    # フィールドごとに平均値と標準偏差を計算
    mu = np.mean(X, axis=0)
    sigma = np.std(X, axis=0)
    
    for i in range(col):
        X[:, i] = (X[:, i] - mu[i]) / sigma[i]
        
    return X

In [ ]:
def gaussian(x, mean, cov):
    """
    多変量ガウス関数
    """
    sample_n = x.shape[0]
    coef1 = 1 / ((2 * np.pi) ** (sample_n / 2.0))
    coef2 = 1 / (np.linalg.det(cov) ** 0.5)
    diff = x - mean
    content = - 0.5 * np.dot( np.dot(diff.T, np.linalg.inv(cov)), diff) # 横ベクトルx縦ベクトル
    
    return coef1 * coef2 * np.exp(content)

In [ ]:
def gaussian_loglikelihood(x, mean, cov):
    """
    多変量ガウスの対数尤度
    x [n_sample, k_features]
    mean [n_sample, k_features]
    cov [k_features, k_feautres]
    """
    
    sample_n = x.shape[0]
    coef1 = - (sample_n / 2) * np.log(2 * np.pi)
    coef2 = 0.5 * np.log(np.linalg.det(cov))
    coef3 = -0.5 * np.dot(np.dot((x - mean).T, np.linalg.inv(cov)), (x - mean)) # スカラ
    
    return coef1 + coef2 + coef3

In [ ]:
# logsumexp関数
def logsumexp1(c, coef=1):
    minC = np.min(c)
    print('minC:', minC)
    print("type(minC):", type(minC))
    print("c - minC:", c-minC)
    x = np.exp(c - minC) # アンダーフローを防ぐ もし、maxCを引くならオーバーフローを防げる
    x = coef * np.log(np.sum(x))
    #x = np.sum(x)
    x += minC
    return x

def logsumexp2(c, coef=1):
    maxC = np.max(c)
    print('maxC:', maxC)
    print("c - maxC:", c-maxC)
    x = np.exp(c - maxC) # オーバーフローを防ぐバージョン
    x = coef * np.log(np.sum(x))
    x += maxC
    return x

In [ ]:
# 確率とlog値の数値の対応関係をチェック
# logsumexp関数の動作チェック

r = np.random.rand(8) / 100000000000 # 0~1
print("r:", r)
print()

sum_r = np.sum(r)
print("sum(r):", sum_r)
print()

prob = r / sum_r
print("prob:", prob)
print()

sum_prob = np.sum(prob)
print("sum(prob):", sum_prob)
print()

log_prob = np.log(prob)
print("log_prob:", log_prob)
print()

log_r = np.log(r)
print("log(r):", log_r)
print(type(log_r))
print(log_r.shape)
print()

logsumexp_log_r = logsumexp1(log_r)
print('logsumexp(log_r):', logsumexp_log_r)
print(type(logsumexp_log_r))
print(logsumexp_log_r.shape)
print()

log_prob2 = log_r - logsumexp_log_r
print("log[r/sum(r)]:", log_prob2)
print()
print("exp( log[r/sum(r)] ):", np.exp(log_prob2))
print()
print("sum { exp( log[r/sum(r)] ) }:", np.sum( np.exp(log_prob2) ))


In [ ]:
def merge_gaussian_loglikelihood(X, mean, cov, pi, K):
    """
    混合ガウス分布の対数尤度
    """
    log_likelihood = 0.0
    
    for n in range(len(X)):
        likelihood = 0.0
        for k in range(K):
            likelihood += pi[k] * gaussian(X[n], mean[k], cov[k])
        log_likelihood += np.log(likelihood)
    
    return log_likelihood

In [ ]:
# トレーニングデータのロード
data = np.genfromtxt("./data/faithful.txt")
X = data[:, 0:2]
X = scale(X) # データの標準化
N = len(X) # サンプルサイズ
#print(X[:5])
#print(N)

In [ ]:
# 多変量ガウス分布の対数尤度のチェック
print("gaussian_likelihood:", gaussian(X[0], mean[0], cov[0]))
print("gaussian_loglikelihood:", gaussian_loglikelihood(X[0], mean[0], cov[0]))
print("exp(gaussian_loglikelihood):", np.exp(gaussian_loglikelihood(X[0], mean[0], cov[0])))

In [ ]:
# 平均μ_k, 分散Σ_k, 混合係数π_k を初期化して、対数尤度の初期値を計算
np.random.seed(0)
mean = np.random.rand(K, X.shape[1]) # 0~1
cov = []
for k in range(K):
    cov.append(np.eye(X.shape[1], X.shape[1]))
cov = np.array(cov)
pi = np.random.rand(K)
print("mean:", mean)
print("cov:", cov)
print("pi:", pi)
print("gauss_dist:", gaussian(X[0], mean[0], cov[0]))

# 負担率の空配列を用意
gamma = np.zeros((N, K))

# 対数尤度の初期値を計算
logLike = merge_gaussian_loglikelihood(X, mean, cov, pi, K)
print("log_likelihood:", logLike)

In [ ]:
turn = 0
while True:
    print("turn:{0}, logLike:{1}".format(turn, logLike))
    
    # Eステップ：現在のパラメタを使って、負担率を計算
    for n in range(N):
        '''
        """logsumexp計算を実装していない普通のEステップ"""
        # 分母はkによらないので、最初に1回だけ計算
        denominator = 0.0
        for j in range(K):
            denominator += pi[j] * gaussian(X[n], mean[j], cov[j])
        # 各kについて負担率を計算
        for k in range(K):
            gamma[n][k] = pi[k] * gaussian(X[n], mean[k], cov[k]) / denominator
        '''
        
        """logsumexp実装のEステップ"""
        rK = np.zeros(K)
        for k in range(K):
            rK[k] = np.log(pi[k]) + gaussian_loglikelihood(X[n], mean[k], cov[k])
            
        logsumexp_GammaK = logsumexp1(rK)
        for k in range(K):
            gamma[n][k] = listGammaNK[k] - logsumexp_GammaK
     
    print("gamma[n][k]:", gamma)
                
    # Mステップ：現在の負担率を使って、パラメタを再計算
    for k in range(K):
        
        # N_kを計算する
        Nk = 0.0
        for n in range(N):
            Nk += gamma[n][k]
        
            
        # 平均を再計算
        mean[k] = np.zeros(X.shape[1]) # features=2
        for n in range(N):
            #print("X[n]:\n", X[n])
            mean[k] += gamma[n][k] * X[n]
        mean[k] /= Nk
        
        # 共分散を再計算
        cov[k] = np.zeros((X.shape[1], X.shape[1]))
        for n in range(N):
            diff = X[n] - mean[k]
            # 縦ベクトルx横ベクトル
            cov[k] += gamma[n][k] * np.matrix(diff).reshape(2, 1) * np.matrix(diff).reshape(1,2)
        cov[k] /=Nk
        
        # 混合係数を再計算
        pi[k] = Nk / N
        
    # 収束判定
    new_logLike = log_likelihood(X, mean, cov, pi)
    diff_logLike = new_logLike - logLike
    if diff_logLike < 0.01:
        break
    logLike = new_logLike
    turn += 1
    
# ガウス分布の平均を描画
for k in range(K):
    plt.scatter(mean[k, 0], mean[k, 1], c='r', marker='o')
    
# 等高線を描画
xlist = np.linspace(-2.5, 2.5, 50)
ylist = np.linspace(-2.5, 2.5, 50)
#x, y = np.meshgrid(xlist, ylist)
pos = np.dstack((xlist, ylist))
for k in range(K):
    #z = bivariate_normal(x, y, 
    #                     np.sqrt(cov[k, 0, 0]), 
    #                     np.sqrt(cov[k, 1, 1]),
    #                    mean[k, 0],
    #                    mean[k, 1],
    #                    cov[k, 0, 1])
    z = stats.multivariate_normal(mean[k], cov[k]).pdf(pos)
    
    cs = contour(x, y, z, 3, colors='k', linewidths=1)
    
# トレーニングデータを描画
plt.plot(X[:, 0], X[:, 1], 'gx')
xlim(-2.5, 2.5)
ylim(-2.5, 2.5)
plt.show()